<a href="https://colab.research.google.com/github/NelisLima/data_science/blob/main/7_agente_ia_analise_acao_mercado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Agentes de IA para Análise de Inteligência de Mercado

Esse projeto tem como objetivo desenvolver um time de agentes de IA para realizarem uma análise de mercado sobre uma empresa, nesse exemplo vamos utilizar a Apple.

Importando bibliotecas necessárias

In [ ]:
import os
import socket
from dotenv import load_dotenv
# ! pip install phidata==2.7.10
from phi.agent import Agent
# ! pip install groq==0.16.0
from phi.model.groq import Groq
# ! pip install duckduckgo_search==7.3.0
from phi.tools.duckduckgo import DuckDuckGo
from phi.tools.yfinance import YFinanceTools

In [ ]:
# Verificação antes de carregar variáveis de ambiente
if os.path.exists('.env'):
    load_dotenv()
    print("[DSA-INFO] Variáveis de ambiente carregadas.")
else:
    print("[DSA-ALERTA] Arquivo .env não encontrado. Algumas configurações podem estar ausentes.")

[DSA-INFO] Variáveis de ambiente carregadas.


In [ ]:
# Verificação de carregamento das variáveis de ambiente
groq_api_key = os.getenv("GROQ_API_KEY")
if groq_api_key:
    print(f"[DSA-INFO] GROQ_API_KEY carregada com sucesso. (Primeiros 5 caracteres: {groq_api_key[:5]}*****)")
else:
    print("[DSA-ERRO] GROQ_API_KEY não encontrada nas variáveis de ambiente. Verifique seu arquivo .env.txt.")

In [ ]:
# Função para verificar conexão com a internet
def dsa_verificar_conexao():
    try:
        socket.create_connection(("www.google.com", 80), timeout=5)
        print("[DSA-INFO] Conexão com a internet verificada.")
        return True
    except OSError:
        print("[DSA-ERRO] Sem conexão com a internet! Algumas ferramentas podem não funcionar.")
        return False

In [ ]:
# Executa a função
conexao_estabelecida = dsa_verificar_conexao()

[DSA-INFO] Conexão com a internet verificada.


In [ ]:
# Criando Agentes de IA
print("[DSA-INFO] Inicializando agentes...")

[DSA-INFO] Inicializando agentes...


In [ ]:
# Define o Agente Financeiro
dsa_agente_financeiro = Agent(name="Analista Financeiro",
                              model=Groq(id="openai/gpt-oss-20b"),
                              tools=[YFinanceTools(stock_price=True,
                                                   analyst_recommendations=True,
                                                   stock_fundamentals=True)] if conexao_estabelecida else [],  # Evita erro se não houver conexão
                              show_tool_calls=True,
                              markdown=True,
                              instructions=["Crie tabelas para comparações",
                                            "Gere os resultados com o idioma Português do Brasil"])

DEBUG    Function: get_current_stock_price registered with yfinance_tools

DEBUG    Function: get_stock_fundamentals registered with yfinance_tools

DEBUG    Function: get_analyst_recommendations registered with yfinance_tools

DEBUG    *********** Agent ID: dd83e2f4-c2a2-4359-8f51-8e9ccdb08e65 ***********

DEBUG    *********** Session ID: 728a3632-b169-4c95-994d-7f8d7acd3b0a ***********

In [ ]:
# Define o Agente de Pesquisa
dsa_agente_pesquisa = Agent(name="Pesquisador Web",
                            model=Groq(id="openai/gpt-oss-20b"),
                            tools=[DuckDuckGo()] if conexao_estabelecida else [],  # Evita erro se não houver conexão
                            show_tool_calls=True,
                            markdown=True,
                            instructions=["Inclua sempre as fontes das informações que você coleta",
                                          "Gere os resultados com o idioma Português do Brasil",
                                          "Use fontes confiáveis, você é um Analista de Pesquisa Sênior"])

In [ ]:
# Define o Time de Agentes
dsa_time_agentes = Agent(team=[dsa_agente_financeiro, dsa_agente_pesquisa],
                         model=Groq(id="openai/gpt-oss-20b"),
                         show_tool_calls=True,
                         markdown=True,
                         instructions=["Inclua sempre a fonte das informações coletadas",
                                       "Crie tabelas para comparações",
                                       "Gere os resultados com o idioma Português do Brasil"],
                         debug_mode=True)

DEBUG    Debug logs enabled

In [ ]:
print("[DSA-INFO] Agentes prontos para análise.")

[DSA-INFO] Agentes prontos para análise.


In [ ]:
# Executando a análise
print("[DSA-INFO] Obtendo recomendações e informações sobre a Apple...")

[DSA-INFO] Obtendo recomendações e informações sobre a Apple...


In [ ]:
# Cria a variável de resposta
response = ""

In [ ]:
# Importando biblioteca para limpar output, na próxima célula e manter apenas informações importantes
from IPython.display import clear_output

In [ ]:
# Captura a resposta correta
for delta in dsa_time_agentes.run("Resumir as recomendações dos analistas sobre investimentos e compartilhar as informações mais recentes sobre a Apple."):
    if isinstance(delta, tuple) and delta[0] == "content":  # Garante que estamos pegando apenas o conteúdo relevante
        response += delta[1]  # O texto relevante está na segunda posição da tupla

clear_output() # limpar demais informações
print(response) # reornando apenas a análise


Running:
 - transfer_task_to_analista_financeiro(additional_information=AAPL, expected_output=Tabela em markdown com as colunas: Recomendações, Número de analistas, Média de preço alvo (USD)., task_description=...)


Running:
 - transfer_task_to_pesquisador_web(additional_information=Apple Inc. (AAPL), últimas 30 dias, expected_output=Lista em markdown com colunas Data, Título, Resumo., task_description=...)

## 1️⃣ Recomendações dos Analistas – Apple (AAPL)

| **Recomendação** | **Número de Analistas** | **Média de Preço‑Alvo (USD)** |
|------------------|------------------------|------------------------------|
| Compra (Strong Buy + Buy) | 29 | N/A* |
| Manutenção (Hold) | 15 | N/A* |
| Venda (Sell + Strong Sell) | 4 | N/A* |

> **Observação**: A API `get_analyst_recommendations` retorna apenas a contagem das recomendações, mas não a média de preço‑alvo. Para obter esse dado seria necessário consultar outras bases de dados financeiras ou relatórios de analistas.  
> **Fonte**: `get_